# MPI for Sonar simulation

## Imports

In [1]:
import ipyparallel as ipp
c = ipp.Client(profile='mpi')

In [2]:
%%px --no-stream --group-outputs=engine

from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print(f"Hi, I'm rank {rank}")

[stdout:0] 
Hi, I'm rank 0
[stdout:1] 
Hi, I'm rank 1


In [3]:
%%px --no-stream --group-outputs=engine

import sys
import os
import matplotlib.pyplot as plt
import numpy as np

sys.path.insert(0, os.path.abspath('/home/hajta2/sonar/sonar-FWI/cli/'))
from simulation.sonar import Sonar
from simulation.utils import EllipsisBottom, gen_velocity_profile
from simulation.sources import GaborSource

%px: 100%|██████████| 2/2 [00:04<00:00,  2.07s/tasks]


In [4]:
%%px --no-stream --group-outputs=engine
from devito import configuration
configuration['mpi'] = True
configuration['language'] = 'openacc'
configuration['platform'] = 'nvidiaX'
configuration['log-level'] = 'DEBUG'
os.environ['DEVITO_ARCH'] = 'nvc'

## Initialization

In [5]:
%%px --no-stream --group-outputs=engine

domain_size = (60, 30)
v_env = 1.5
ns = 128
source_distance = 0.002
f0 = 50
space_order = 8
spatial_dist = round(v_env / f0 / 3, 3)
dt = spatial_dist / 20

In [6]:
%%px --no-stream --group-outputs=engine

domain_dims = (
    round(domain_size[0] / spatial_dist),
    round(domain_size[1] / spatial_dist),
)
vp = gen_velocity_profile(EllipsisBottom(True), domain_dims, spatial_dist, v_water = v_env)

%px: 100%|██████████| 2/2 [00:11<00:00,  5.67s/tasks]


[stderr:0] 
51it [00:12,  3.96it/s]
[stderr:1] 
51it [00:08,  6.22it/s]


In [8]:
%%px --no-stream --group-outputs=engine

from examples.seismic import Model
model = Model(
    vp=vp,
    origin=(0.0, 0.0),
    spacing=(spatial_dist, spatial_dist),
    shape=domain_dims,
    space_order=space_order,
    nbl=(ns - 1) / 2 * source_distance / dt,
    bcs="damp",
    dt=dt,
    dtype=np.float64,
)

%px: 100%|██████████| 2/2 [00:05<00:00,  2.86s/tasks]


[stderr:None] 
Operator `initdamp` generated in 1.60 s
  * lowering.Clusters: 0.79 s (49.7 %)
     * specializing.Clusters: 0.58 s (36.5 %)
        * fuse: 0.32 s (20.2 %)
  * lowering.IET: 0.60 s (37.8 %)
     * specializing.IET: 0.46 s (29.0 %)
Flops reduction after symbolic optimization: [72 --> 72]
Operator `initdamp` fetched `/tmp/devito-jitcache-uid543800090/8d9e41fc7b2cd6b76f6b35ddf7489f2d48c80977.c` in 0.50 s from jit-cache
Operator `initdamp` generated in 0.67 s
  * lowering.Clusters: 0.35 s (53.0 %)
     * specializing.Clusters: 0.27 s (40.9 %)
        * fuse: 0.16 s (24.3 %)
  * lowering.IET: 0.23 s (34.9 %)
     * specializing.IET: 0.18 s (27.3 %)
Flops reduction after symbolic optimization: [72 --> 72]
Operator `initdamp` fetched `/tmp/devito-jitcache-uid543800090/8d9e41fc7b2cd6b76f6b35ddf7489f2d48c80977.c` in 0.25 s from jit-cache


CompositeError: one or more exceptions raised in: %px
[Engine Exception]EngineError: Engine 0 died while running task 'f6b86f14-c60c2d8f1f99833150730b3e_209416_13'
[Engine Exception]EngineError: Engine 1 died while running task 'f6b86f14-c60c2d8f1f99833150730b3e_209416_14'